**SOW-MKI49: Neural Information Processing Systems**  
*Weeks 4 and 5: Assignment (225 points + 30 bonus points)*  
Author: Umut

In [14]:
# Group number: 24
# Franka Buytenhuijs, s4356845
# Hugo Chateau-Laurent, s1023970
# Maria Tsfasman, s1021505

In [15]:
from chainer import ChainList, optimizers, serializers
import chainer
import chainer.functions as F
import chainer.links as L
import numpy as np

# start
# end

**WaveNet component (75 points)**

* Implement missing parts of the call method (y and z). **25 points**
* Implement residual block class. **50 points**

---
Reminder:

* One convolution layer that has 61 kernels of size 2 with no nonlinearities.
![alt text](http://i67.tinypic.com/21mgi2w.png)
![alt text](http://i67.tinypic.com/292n04y.png)
---



In [30]:
# The output of the first layer is the input of the first block. 
# Each block has two outputs. Let's call them A and B. A is the input of the next block. 
# B is summed with B of other blocks. A of last block is discarded, and ΣB is the input 
# of the first layer after the last block.

class _WaveNet(ChainList):
    def __init__(self):
        links = (L.Convolution2D(61, 61, (1, 2)),)
        links += tuple(_ResidualBlock((1, 2 ** (i % 6))) for i in range(6))
        links += (L.Convolution2D(512, 512, 1),)
        links += (L.Convolution2D(512, 3843, 1),) # two parallel in one -> use split

        super(_WaveNet, self).__init__(*links)

    def __call__(self, x):
        y = (self[0](F.pad(x, ((0, 0), (0, 0), (0, 0), (1, 0)), 'constant')),) # first layer output
        z = 0

        for i in range(1, len(self) - 2):
            y = self[i](y[0]) # call block with output from first parallel layer of previous block
            z += y[1] # sum output of second parallel layers of all blocks
        
        z = self[7](z) # output of second parallel layer of last block as input
        z = F.relu(z)
        k, psi_u = F.split_axis(self[8](z), (3721,), 1) # pairwise=k and unary=psi-u on lecture slide 26

        return F.reshape(y, (y.shape[0], 61, 61, y.shape[3])), \
               F.reshape(z, (z.shape[0], 2, 61, z.shape[3]))

class ResidualBlock(ChainList):
    def __init__(self, dilation):
        super(ResidualBlock, self).__init__()

        with self.init_scope():
            links = (L.DilatedConvolution2D(61, 122, (1, 2), dilate = dilation),
                     L.Convolution2D(61, 61, 1),
                     L.Convolution2D(61, 512, 1),)

    def __call__(self, x, finetune = False):
        # start 2
        #gated activation unit (taken from slide 17+18)
        #layer = L.(Dilated)Convolution2D(..., 2 x out_channels, ...)
        #h = F.split_axis(layer(x), 2, 1)
        #y = F.sigmoid(h[0]) * F.tanh(h[1])
        
        # first layer
        y = F.split_axis(self[0](F.pad(x, ((0, 0), (0, 0), (0, 0), (self[0].dilate[1], 0)), 'constant')), 2, 1)
        y = F.sigmoid(y[0]) * F.tanh(y[1]) # gated activation unit
        
        # parallel layers
        y1 = self[1](y)        
        y2 = self[2](y)
        
        return x + y1, y2
    
        # end

        return y

**CRF-RNN component (50 points)**

* Implement missing parts of the call method (z). **25 points**
* Why is z not normalized in the last iteration? **25 points**

---

Reminder:

![alt text](http://i68.tinypic.com/sy6mix.png)

---

In [31]:
class _CRF(ChainList):
    def __init__(self):
        super(_CRF, self).__init__(L.ConvolutionND(1, 2, 2, 1, nobias = True))

    def __call__(self, k, psi_u):
        Q = psi_u

        for i in range(5): #input for first layer after first 4 iterations, and output after 5th iteration
            
            # message passing layer
            Q = F.batch_matmul(Q,k)
            Q.shape # should be N × 2 x 61 tensor
            
            # compatibility transform layer
            Q = self[0](Q)
            Q = -psi_u-Q
            Q.shape # should be N × 2 x 61 tensor
            
            # local update and normalization layer            
            if i < 4:
                Q = F.softmax(Q)
                Q.shape # should be N × 2 x 61 tensor
        return z
#Question: We normalize over z to get the conditional probability p(y|x). 
#In the last layer we dont need that anymore... (make more specific)

**WaveCRF model (50 points)**

1. Implement missing parts of the call method (k, psi_u and Q_hat). **20 points**
2. Implement missing parts of the save and load methods (save and load model). **10 points**
3. Implement missing parts of the test and train methods (forward and/or backward propagate). **20 points**

In [32]:
class WaveCRF(object):
    def __init__(self):
        self.log = {('test', 'accuracy'): (), ('test', 'loss'): (), ('training', 'accuracy'): (),
                    ('training', 'loss'): ()}
        self.model = ChainList(_WaveNet(), _CRF())
        self.optimizer = optimizers.Adam(0.0002, 0.5)

        self.optimizer.setup(self.model)

    def __call__(self, x):
        k, psi_u = self.model[0](x)
        Q_hat = self.model[1](k, psi_u)

        return F.transpose(F.reshape(Q_hat, (x.shape[0], x.shape[3], 2, 61)), (0, 2, 3, 1))

    @classmethod
    def load(cls, directory):
        self = cls()
        self.log = np.load('{}/log.npy'.format(directory))

        serializers.load_npz('{}/weights.npz'.format(directory), self.model)
        serializers.load_npz('{}/optimizer.npz'.format(directory), self.optimizer)

        return self

    def save(self, directory):
        np.save('{}/log.npy'.format(directory), self.log)
        serializers.save_npz('{}/weights.npz'.format(directory), self.model)
        serializers.save_npz('{}/optimizer.npz'.format(directory), self.optimizer)

    def test(self, Q, x):
        with chainer.using_config('train', False):
            # Forward prop
            # Forward prop

            self.log['test', 'accuracy'] += (float(F.accuracy(Q_hat, Q).data),)
            self.log['test', 'loss'] += (float(loss.data),)

    def train(self, Q, x):
        # Forward prop
        y = self(x)
        # Forward prop
        loss = F.softmax_cross_entropy(y, Q)
        # Backprop start
        self.model.cleargrads()
        loss.backward()
        self.optimizer.update()
        # Backprop end
        

        self.log['training', 'accuracy'] += (float(F.accuracy(Q_hat, Q).data),)
        self.log['training', 'loss'] += (float(loss.data),)

In [33]:
%matplotlib inline

import IPython
import chainer
import matplotlib
import numpy
import os
import pickle
import random
import tqdm

# start
from zipfile import ZipFile
import bz2

# end 

In [34]:
batch_size = 30
epochs = 100
root = '.'

In [35]:

# start
def save(filename, myobj):
    """
    save object to file using pickle
    
    @param filename: name of destination file
    @type filename: str
    @param myobj: object to save (has to be pickleable)
    @type myobj: obj
    """

    try:
        f = bz2.BZ2File(filename, 'wb')
    except (IOError, details) as err:
        sys.stderr.write('File ' + filename + ' cannot be written\n')
        sys.stderr.write(err[1])
        return

    pickle.dump(myobj, f, protocol=2)
    f.close()
    
def load(filename):
    """
    Load from filename using pickle
    
    @param filename: name of file to load from
    @type filename: str
    """

    try:
        f = bz2.BZ2File(filename, 'rb')
    except (IOError, details) as err:
        sys.stderr.write('File ' + filename + ' cannot be read\n')
        sys.stderr.write(err[1])
        return

    myobj = pickle.load(f)
    f.close()
    return myobj

save = False
if save:
    with open('{}/Data/piano_rolls.p'.format(root), 'rb') as f:
        piano_rolls = pickle.load(f)
        save('{}/Data/compressed_piano_rolls.p'.format(root), piano_rolls)
piano_rolls = load('{}/Data/compressed_piano_rolls.p'.format(root))
# end

        
'''
# for file
with open('{}/Data/piano_rolls.p'.format(root), 'rb') as f:
    piano_rolls = pickle.load(f)
'''

keys = sorted(piano_rolls.keys())

random.seed(6)
random.shuffle(keys)

test_set = dict((key, piano_rolls[key]) for key in keys[:int(0.1 * len(keys))])
training_set = dict((key, piano_rolls[key]) for key in keys[int(0.1 * len(keys)):])
training_set_keys = list(training_set.keys())

In [36]:
waveCRF = WaveCRF()

waveCRF.model.to_gpu()

TypeError: super(type, obj): obj must be an instance or subtype of type

In [0]:
for epoch in tqdm.tnrange(epochs):
    random.shuffle(training_set_keys)

    batch = ()

    for key in tqdm.tqdm_notebook(training_set_keys, leave = False):
        i = random.randint(0, training_set[key].shape[1] - 80)
        batch += (training_set[key][32 : 93, i : i + 80],)

        if len(batch) == batch_size:
            batch = waveCRF.model.xp.array(batch)

            waveCRF.train(batch[:, :, 1:].astype('i'), batch[:, :, None, :-1].astype('f'))

            batch = ()

    for key in tqdm.tqdm_notebook(test_set, leave = False):
        batch = waveCRF.model.xp.array((test_set[key][32 : 93],))

        waveCRF.test(batch[:, :, 1:].astype('i'), batch[:, :, None, :-1].astype('f'))

    IPython.display.clear_output()

    for i, key in enumerate(waveCRF.log):
        matplotlib.pyplot.subplot(221 + i)
        matplotlib.pyplot.plot(numpy.array(waveCRF.log[key]).reshape(epoch + 1, -1).mean(1))
        matplotlib.pyplot.xlabel('iteration')
        matplotlib.pyplot.ylabel(key)

    matplotlib.pyplot.tight_layout()
    matplotlib.pyplot.show()
    os.makedirs('{}/Models/WaveCRF/{}'.format(root, epoch))
    waveCRF.save('{}/Models/WaveCRF/{}'.format(root, epoch))

**Test (50 points)**  

* Generate a number of samples, pick the best one and play it in the notebook. **50 points**

In [0]:
# Test

**Bonus question (30 points)**

* Discuss how you can improve the model (you can talk about different architectures or different ways to encode the inputs, etc.) **10 points**
* Discuss the assumptions behind the meanfield approximation and its shortcomings. **10 points**
* Prove that the iterative update equation (CRF-RNN component) is differentiable so that we can backpropagate through them. **10 points**